In [63]:


import os

from google.cloud import bigquery
import pandas as pd

import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output

# ── 1) GCP / BigQuery setup ─────────────────────────────────────────────
# adjust this to wherever your JSON key lives
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "mindful-vial-460001-h6-4d83b36dd3e9.json"

PROJECT = "mindful-vial-460001-h6"
DATASET = "euphoria"
client = bigquery.Client(project=PROJECT)

# ── 2) One-shot “all KPIs” union query ────────────────────────────────────
SQL = f"""
WITH
  viewed AS (
    SELECT country, SUM(length) AS total_watch_seconds
    FROM `{PROJECT}.{DATASET}.watch_topic`
    GROUP BY country
    ORDER BY total_watch_seconds DESC
    LIMIT 10
  ),
  purchased AS (
    SELECT product_name, COUNT(*) AS purchase_count
    FROM `{PROJECT}.{DATASET}.purchase_events_topic`
    GROUP BY product_name
    ORDER BY purchase_count DESC
    LIMIT 8
  ),
  streamer_perf AS (
    SELECT
      p.screen_name,
      SUM((s.viewers_total/NULLIF(s.length,0)) * s.comments_total) AS performance_score
    FROM `{PROJECT}.{DATASET}.streams_topic`   AS s
    JOIN `{PROJECT}.{DATASET}.partners_topic`  AS p
      ON s.partner_id = p.partner_id
    GROUP BY p.screen_name
    ORDER BY performance_score DESC
    LIMIT 10
  ),
  best_games AS (
    SELECT product_name, COUNT(*) AS purchase_count
    FROM `{PROJECT}.{DATASET}.purchase_events_topic`
    WHERE category = 'game'
    GROUP BY product_name
    ORDER BY purchase_count DESC
    LIMIT 2
  ),
  streamed_games AS (
    SELECT
      g.title      AS game_title,
      COUNT(*)     AS stream_count
    FROM `{PROJECT}.{DATASET}.streams_topic` AS s
    JOIN `{PROJECT}.{DATASET}.games_topic`   AS g
      ON s.game_id = g.game_id
    GROUP BY g.title
    ORDER BY stream_count DESC
    LIMIT 2
  ),
  top_cust_purch AS (
    SELECT customer_id, COUNT(*) AS purchase_count
    FROM `{PROJECT}.{DATASET}.purchase_events_topic`
    GROUP BY customer_id
    ORDER BY purchase_count DESC
    LIMIT 1000
  ),
  top_cust_watch AS (
    SELECT customer_id, SUM(length) AS total_watch_seconds
    FROM `{PROJECT}.{DATASET}.watch_topic`
    GROUP BY customer_id
    ORDER BY total_watch_seconds DESC
    LIMIT 1000
  ),
  monthly_watch AS (
    SELECT
      FORMAT('%04d-%02d',
        EXTRACT(YEAR  FROM DATE(date)),
        EXTRACT(MONTH FROM DATE(date))
      ) AS period,
      SUM(length) AS total_watch_seconds
    FROM `{PROJECT}.{DATASET}.watch_topic`
    GROUP BY period
    ORDER BY period
  ),
  yearly_watch AS (
    SELECT
      CAST(EXTRACT(YEAR FROM DATE(date)) AS STRING) AS period,
      SUM(length) AS total_watch_seconds
    FROM `{PROJECT}.{DATASET}.watch_topic`
    GROUP BY period
    ORDER BY period
  ),
  monthly_merch AS (
    SELECT
      FORMAT('%04d-%02d',
        EXTRACT(YEAR  FROM DATE(timestamp)),
        EXTRACT(MONTH FROM DATE(timestamp))
      ) AS period,
      SUM(price) AS total_merch_sales
    FROM `{PROJECT}.{DATASET}.purchase_events_topic`
    WHERE category = 'merch'
    GROUP BY period
    ORDER BY period
  ),
  yearly_merch AS (
    SELECT
      CAST(EXTRACT(YEAR FROM DATE(timestamp)) AS STRING) AS period,
      SUM(price) AS total_merch_sales
    FROM `{PROJECT}.{DATASET}.purchase_events_topic`
    WHERE category = 'merch'
    GROUP BY period
    ORDER BY period
  )

SELECT 'Top 10 Viewed Countries'       AS kpi, country        AS label, CAST(total_watch_seconds AS STRING)     AS value FROM viewed
UNION ALL
SELECT 'Top 8 Purchased Products'      AS kpi, product_name   AS label, CAST(purchase_count            AS STRING) AS value FROM purchased
UNION ALL
SELECT 'Top 10 Streamer Performance'   AS kpi, screen_name    AS label, CAST(ROUND(performance_score,2) AS STRING) AS value FROM streamer_perf
UNION ALL
SELECT 'Top 2 Best-Selling Games'      AS kpi, product_name   AS label, CAST(purchase_count            AS STRING) AS value FROM best_games
UNION ALL
SELECT 'Top 2 Most-Streamed Games'     AS kpi, game_title     AS label, CAST(stream_count              AS STRING) AS value FROM streamed_games
UNION ALL
SELECT 'Top 1000 Customers by Purchases'      AS kpi, customer_id AS label, CAST(purchase_count            AS STRING) AS value FROM top_cust_purch
UNION ALL
SELECT 'Top 1000 Customers by Watch Seconds'  AS kpi, customer_id AS label, CAST(total_watch_seconds       AS STRING) AS value FROM top_cust_watch
UNION ALL
SELECT 'Monthly Watch (sec)'           AS kpi, period         AS label, CAST(total_watch_seconds       AS STRING) AS value FROM monthly_watch
UNION ALL
SELECT 'Yearly Watch (sec)'            AS kpi, period         AS label, CAST(total_watch_seconds       AS STRING) AS value FROM yearly_watch
UNION ALL
SELECT 'Monthly Merch Sales'           AS kpi, period         AS label, FORMAT('$%.2f', total_merch_sales)        AS value FROM monthly_merch
UNION ALL
SELECT 'Yearly Merch Sales'            AS kpi, period         AS label, FORMAT('$%.2f', total_merch_sales)        AS value FROM yearly_merch
;
"""

# load it once
df = client.query(SQL).to_dataframe()

# ── 3) Dash app ─────────────────────────────────────────────────────────
app = dash.Dash(__name__)
app.title = "Euphoria Dynamic KPI Dashboard"

app.layout = html.Div([
    html.H1("Euphoria KPI Dashboard"),
    dcc.Dropdown(
        id="kpi-dropdown",
        options=[{"label": k, "value": k} for k in df["kpi"].unique()],
        value=df["kpi"].unique()[0],
        clearable=False,
        style={"width":"60%", "marginBottom":"1em"}
    ),
    dash_table.DataTable(
        id="kpi-table",
        columns=[{"name": c, "id": c} for c in df.columns],
        data=[],
        page_size=20,
        style_table={"overflowX":"auto"},
        style_header={"fontWeight":"bold"},
        style_cell={"textAlign":"left", "padding":"5px"}
    )
], style={"padding":"20px"})


@app.callback(
    Output("kpi-table", "data"),
    Input("kpi-dropdown", "value")
)
def filter_table(selected_kpi):
    dff = df[df["kpi"] == selected_kpi]
    return dff.to_dict("records")


if __name__ == "__main__":
    app.run(debug=True)




# Live Watch Hours by Country

In [57]:
import os
from google.cloud import bigquery
import pandas as pd

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# ── 1) Set up your GCP credentials & BQ client ────────────────────────────────
# either set GOOGLE_APPLICATION_CREDENTIALS in your env, or do it in code:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/your-key.json"
PROJECT = "mindful-vial-460001-h6"
DATASET = "euphoria"

bq = bigquery.Client(project=PROJECT)

# ── 2) Dash app boilerplate ────────────────────────────────────────────────────
app = dash.Dash(__name__)
app.title = "Live Watch Hours by Country"

app.layout = html.Div([
    html.H1("Live Watch Hours by Country (last 5 minutes)", style={"textAlign":"center"}),
    dcc.Graph(id="live-choropleth"),
    dcc.Interval(id="interval", interval=10*1000, n_intervals=0)  # every 10 seconds
])

# ── 3) Callback to refresh the map ─────────────────────────────────────────────
@app.callback(
    Output("live-choropleth", "figure"),
    Input("interval", "n_intervals")
)
def update_map(n):
    sql = f"""
      SELECT
        country,
        SUM(length) / 3600.0 AS watch_hours
      FROM `{PROJECT}.{DATASET}.watch_topic`
      WHERE date >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 5 MINUTE)
      GROUP BY country
    """
    df = bq.query(sql).to_dataframe()
    if df.empty:
        # show an empty frame
        return px.choropleth(
            locations=[],
            locationmode="country names",
            color=[],
            title="No data yet"
        )
    fig = px.choropleth(
        df,
        locations="country",
        locationmode="country names",
        color="watch_hours",
        hover_name="country",
        color_continuous_scale="Viridis",
        range_color=(0, df.watch_hours.max()),
        labels={"watch_hours":"Hours watched"}
    )
    fig.update_layout(
        margin={"r":0,"t":30,"l":0,"b":0},
        coloraxis_colorbar={"title":"Hours"}
    ) 
    return fig

# ── 4) Run it ───────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    app.run(debug=True)


# Big Query SQL Runner

In [56]:
import os
import re
from dash import Dash, html, dcc, Input, Output
import dash_table
from google.cloud import bigquery

# -----------------------------------------------------------------------------
# CONFIG — update as needed
# -----------------------------------------------------------------------------
# If running locally, uncomment and set your service account:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/your/creds.json"

PROJECT = "mindful-vial-460001-h6"
DATASET = "euphoria"

# -----------------------------------------------------------------------------
# CLIENT SETUP
# -----------------------------------------------------------------------------
client = bigquery.Client(project=PROJECT)

# -----------------------------------------------------------------------------
# DASH APP
# -----------------------------------------------------------------------------
app = Dash(__name__)
app.layout = html.Div([
    html.H2("BigQuery SQL Runner"),
    dcc.Textarea(
        id='sql-input',
        value=(
            f"SELECT *\n"
            f"FROM `{PROJECT}.{DATASET}.streams_topic`\n"
            "LIMIT 5;"
        ),
        style={'width': '100%', 'height': '120px'}
    ),
    html.Button("Run Query", id='run-btn', n_clicks=0, style={'marginTop': '10px'}),
    html.Div(id='table-container', style={'marginTop': '20px'})
])

@app.callback(
    Output('table-container', 'children'),
    Input('run-btn', 'n_clicks'),
    Input('sql-input', 'value')
)
def render_table(n_clicks, query):
    # Don't run until button clicked
    if not n_clicks:
        return html.Div("Enter a SQL query above and click ▶️ Run Query")

    # Replace placeholders if used
    q = re.sub(r"\bPROJECT\b", PROJECT, query)
    q = re.sub(r"\bDATASET\b", DATASET, q)

    # Execute and fetch DataFrame
    df = client.query(q).to_dataframe(create_bqstorage_client=False)

    # Render Dash DataTable
    return dash_table.DataTable(
        columns=[{'name': c, 'id': c} for c in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_header={'fontWeight': 'bold'},
        style_cell={'textAlign': 'left'}
    )

if __name__ == "__main__":
    app.run(debug=True)


# Watch Hours by Country Heat Map

In [55]:
import os
from datetime import datetime
from google.cloud import bigquery

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# ── 1) GCP / BigQuery setup ─────────────────────────────────────────────
PROJECT = "mindful-vial-460001-h6"
DATASET = "euphoria"
bq = bigquery.Client(project=PROJECT)

# ── 2) Build list of the last 10 years for the dropdown ─────────────────
current_year = datetime.utcnow().year
YEARS = list(range(current_year - 9, current_year + 1))[::-1]  # e.g. [2025,2024,…,2016]

# ── 3) Dash app ────────────────────────────────────────────────────────
app = dash.Dash(__name__)
app.title = "Yearly Relative Watch Hours"

app.layout = html.Div([
    html.H1("Watch Hours by Country (Relative Rank)"),
    html.Div([
      html.Label("Select Year:"),
      dcc.Dropdown(
          id='year-dropdown',
          options=[{'label': str(y), 'value': y} for y in YEARS],
          value=current_year,
          clearable=False,
          style={'width':'120px'}
      )
    ], style={'marginBottom':'1em'}),
    dcc.Graph(id='choropleth'),
])

# ── 4) Callback: fetch & plot the relative percent‐rank choropleth ────────
@app.callback(
    Output('choropleth', 'figure'),
    Input('year-dropdown', 'value'),
)
def update_choropleth(selected_year):
    # 1) Compute each country's watch_hours and its percentile rank within that year
    sql = f"""
    WITH country_totals AS (
      SELECT
        country,
        SUM(length)/3600.0 AS watch_hours
      FROM `{PROJECT}.{DATASET}.watch_topic`
      WHERE EXTRACT(YEAR FROM DATE(date)) = {selected_year}
      GROUP BY country
    )
    SELECT
      country,
      watch_hours,
      PERCENT_RANK() OVER (ORDER BY watch_hours) AS pct_rank
    FROM country_totals
    """
    df = bq.query(sql).to_dataframe()

    # 2) If no data, show empty
    if df.empty:
        return px.choropleth(
            locations=[],
            color=[]
        )

    # 3) Plot using pct_rank as the color
    fig = px.choropleth(
        df,
        locations="country",
        locationmode="country names",
        color="pct_rank",
        hover_name="country",
        hover_data={"watch_hours":":.1f", "pct_rank":":.2f"},
        color_continuous_scale="Viridis",
        range_color=(0,1),
        labels={"pct_rank":"Relative rank","watch_hours":"Hours"},
        title=f"Relative Watch Hours in {selected_year}"
    )
    fig.update_layout(
        margin={"r":0,"t":40,"l":0,"b":0},
        coloraxis_colorbar={"title":"Percentile"}
    )
    return fig

# ── 5) Main ─────────────────────────────────────────────────────────────
if __name__ == "__main__":
    app.run(debug=True)


C:\Users\Armstrong\AppData\Local\Temp\ipykernel_3572\25145575.py:16: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).

